This notebook is based on the tensorflow docs: 


https://www.tensorflow.org/tensorboard/hyperparameter_tuning_with_hparams

adapted for a regression problem.

In [1]:
%load_ext tensorboard
# Clear any logs from previous runs
!rm -rf ./logs/

In [2]:
import tensorflow as tf
from tensorboard.plugins.hparams import api as hp
import numpy as np
from sklearn.preprocessing import StandardScaler

In [35]:
# file = "/home/isidro/chains_server/owaCDM_phy_HD+SN+BBAO+Planck_nested_dynesty_multi_1.txt"
file = "/home/isidro/SimpleMC/chains/LCDM_phy_HD+SN+BBAO+Planck_nested_dynesty_multi_1.txt"

split = 0.8
epochs = 100
nlayers = 2
batch_size = 0

data = np.loadtxt(file, usecols=(1,2,3,4,5,6))
params = data[7000:7500, 1:]
logL = data[7000:7500, 0]
print("len original data {}".format(len(data)))

nparams = len(params)
randomize = np.random.permutation(nparams)
params = params[randomize]
logL = logL[randomize]

maxLogL = np.max(logL)
minLogL = np.min(logL)
ntrain = int(split * nparams)
indx = [ntrain]
    # xtrain <- params_training ytrain<-logLtraining
params_training, params_testing = np.split(params, indx)
logL_training, logL_testing = np.split(logL, indx)


# create scaler
scaler = StandardScaler()
# # scaler = MinMaxScaler(feature_range=(-1,1))
# # fit scaler on data
scaler.fit(params_training)
# # apply transform
params_training = scaler.transform(params_training)
params_testing = scaler.transform(params_testing)

print("Total len dataset {}," 
      "len training set {}, len test set {}".format(nparams, 
                                                 len(params_training), len(params_testing)))

# print(n_cols)

len original data 8243
Total len dataset 500,len training set 400, len test set 100


In [36]:
# loss -> val_loss
callbacks = [tf.keras.callbacks.EarlyStopping(monitor='loss', mode='min',
                                   min_delta=0,
                                   patience=5,
                                   restore_best_weights=True)
                                   ]

n_cols = params_training.shape[1]

In [37]:
HP_NUM_UNITS = hp.HParam('num_units', hp.Discrete([50, 100, 150, 200, 250, 300]))
# HP_DROPOUT = hp.HParam('dropout', hp.RealInterval(0.0, 0.2))
# HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'sgd']))
# HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'Adadelta']))
# HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam']))
HP_BATCHSIZE = hp.HParam('batch_size', hp.Discrete([4,8, 16,32,64,128]))


In [38]:
# METRIC_ACCURACY = 'accuracy'

with tf.summary.create_file_writer('logs/hparam_tuning').as_default():
  hp.hparams_config(
#     hparams=[HP_NUM_UNITS, HP_OPTIMIZER, HP_BATCHSIZE],
    hparams=[HP_NUM_UNITS, HP_BATCHSIZE],
    metrics=[hp.Metric('loss', display_name="Loss")])


In [42]:
def train_test_model(hparams):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(hparams[HP_NUM_UNITS], activation=tf.nn.relu, input_shape=(n_cols,)),
#         tf.keras.layers.Dropout(hparams[HP_DROPOUT]),
        tf.keras.layers.Dense(hparams[HP_NUM_UNITS], activation=tf.nn.relu),
#         tf.keras.layers.Dropout(hparams[HP_DROPOUT]),
        tf.keras.layers.Dense(hparams[HP_NUM_UNITS], activation=tf.nn.relu),
#         tf.keras.layers.Dropout(hparams[HP_DROPOUT]),
        tf.keras.layers.Dense(1)
    ])
                              
    model.compile(
#             optimizer=hparams[HP_OPTIMIZER],
            optimizer='adam',
#             loss='mean_squared_error',
            loss='mean_squared_error',
            metrics=['mean_squared_error']
          )
    # Run with 1 epoch to speed things up for demo purposes
    # model.fit(x_train, y_train, epochs=1, callbacks=callbacks) 
    # _, loss = model.evaluate(x_test, y_test)
    model.fit(params_training, logL_training, epochs=epochs, 
              batch_size=hparams[HP_BATCHSIZE], callbacks=callbacks) 
    _, loss = model.evaluate(params_testing, logL_testing)


    return loss
    

In [43]:
def run(run_dir, hparams):
  with tf.summary.create_file_writer(run_dir).as_default():
    hp.hparams(hparams)  # record the values used in this trial
    loss = train_test_model(hparams)
    tf.summary.scalar("loss", loss, step=1)

In [44]:
session_num = 0

for batch_size in HP_BATCHSIZE.domain.values:
    for num_units in HP_NUM_UNITS.domain.values:
#       for dropout_rate in (HP_DROPOUT.domain.min_value, HP_DROPOUT.domain.max_value):
#         for optimizer in HP_OPTIMIZER.domain.values:
          hparams = {
              HP_NUM_UNITS: num_units,
    #               HP_DROPOUT: dropout_rate,
    #               HP_OPTIMIZER: optimizer,
              HP_BATCHSIZE: batch_size
          }
          run_name = "run-%d" % session_num
          print('--- Starting trial: %s' % run_name)
          print({h.name: hparams[h] for h in hparams})
          run('logs/hparam_tuning/' + run_name, hparams)
          session_num += 1


--- Starting trial: run-0
{'num_units': 50, 'batch_size': 4}
Epoch 1/100
100/100 [==============================] - 0s 623us/step - loss: 547.2614 - mean_squared_error: 547.2614
Epoch 2/100
100/100 [==============================] - 0s 529us/step - loss: 45.3141 - mean_squared_error: 45.3141
Epoch 3/100
100/100 [==============================] - 0s 569us/step - loss: 25.4424 - mean_squared_error: 25.4424
Epoch 4/100
100/100 [==============================] - 0s 550us/step - loss: 15.8641 - mean_squared_error: 15.8641
Epoch 5/100
100/100 [==============================] - 0s 528us/step - loss: 11.2332 - mean_squared_error: 11.2332
Epoch 6/100
100/100 [==============================] - 0s 604us/step - loss: 7.8819 - mean_squared_error: 7.8819
Epoch 7/100
100/100 [==============================] - 0s 526us/step - loss: 5.6017 - mean_squared_error: 5.6017
Epoch 8/100
100/100 [==============================] - 0s 561us/step - loss: 3.9752 - mean_squared_error: 3.9752
Epoch 9/100
100/100 [==

100/100 [==============================] - 0s 649us/step - loss: 259.4032 - mean_squared_error: 259.4032
Epoch 2/100
100/100 [==============================] - 0s 658us/step - loss: 27.9325 - mean_squared_error: 27.9325
Epoch 3/100
100/100 [==============================] - 0s 638us/step - loss: 16.6913 - mean_squared_error: 16.6913
Epoch 4/100
100/100 [==============================] - 0s 672us/step - loss: 11.4127 - mean_squared_error: 11.4127
Epoch 5/100
100/100 [==============================] - 0s 643us/step - loss: 6.7836 - mean_squared_error: 6.7836
Epoch 6/100
100/100 [==============================] - 0s 653us/step - loss: 4.5095 - mean_squared_error: 4.5095
Epoch 7/100
100/100 [==============================] - 0s 670us/step - loss: 3.2844 - mean_squared_error: 3.2844
Epoch 8/100
100/100 [==============================] - 0s 650us/step - loss: 1.9296 - mean_squared_error: 1.9296
Epoch 9/100
100/100 [==============================] - 0s 637us/step - loss: 1.2505 - mean_squared

100/100 [==============================] - 0s 927us/step - loss: 0.0626 - mean_squared_error: 0.0626
Epoch 17/100
100/100 [==============================] - 0s 897us/step - loss: 0.0442 - mean_squared_error: 0.0442
Epoch 18/100
100/100 [==============================] - 0s 895us/step - loss: 0.0381 - mean_squared_error: 0.0381
Epoch 19/100
100/100 [==============================] - 0s 871us/step - loss: 0.0359 - mean_squared_error: 0.0359
Epoch 20/100
100/100 [==============================] - 0s 878us/step - loss: 0.0394 - mean_squared_error: 0.0394
Epoch 21/100
100/100 [==============================] - 0s 888us/step - loss: 0.0535 - mean_squared_error: 0.0535
Epoch 22/100
100/100 [==============================] - 0s 920us/step - loss: 0.0252 - mean_squared_error: 0.0252
Epoch 23/100
100/100 [==============================] - 0s 872us/step - loss: 0.0325 - mean_squared_error: 0.0325
Epoch 24/100
100/100 [==============================] - 0s 910us/step - loss: 0.0353 - mean_squared_e

50/50 [==============================] - 0s 537us/step - loss: 0.1480 - mean_squared_error: 0.1480
Epoch 37/100
50/50 [==============================] - 0s 582us/step - loss: 0.1329 - mean_squared_error: 0.1329
Epoch 38/100
50/50 [==============================] - 0s 511us/step - loss: 0.1552 - mean_squared_error: 0.1552
Epoch 39/100
50/50 [==============================] - ETA: 0s - loss: 0.0821 - mean_squared_error: 0.08 - 0s 527us/step - loss: 0.1285 - mean_squared_error: 0.1285
Epoch 40/100
50/50 [==============================] - 0s 556us/step - loss: 0.1143 - mean_squared_error: 0.1143
Epoch 41/100
50/50 [==============================] - 0s 554us/step - loss: 0.0994 - mean_squared_error: 0.0994
Epoch 42/100
50/50 [==============================] - 0s 559us/step - loss: 0.0985 - mean_squared_error: 0.0985
Epoch 43/100
50/50 [==============================] - 0s 581us/step - loss: 0.0967 - mean_squared_error: 0.0967
Epoch 44/100
50/50 [==============================] - 0s 591us/st

50/50 [==============================] - 0s 672us/step - loss: 0.2751 - mean_squared_error: 0.2751
Epoch 17/100
50/50 [==============================] - 0s 743us/step - loss: 0.2027 - mean_squared_error: 0.2027
Epoch 18/100
50/50 [==============================] - 0s 611us/step - loss: 0.1432 - mean_squared_error: 0.1432
Epoch 19/100
50/50 [==============================] - 0s 625us/step - loss: 0.1319 - mean_squared_error: 0.1319
Epoch 20/100
50/50 [==============================] - 0s 621us/step - loss: 0.1012 - mean_squared_error: 0.1012
Epoch 21/100
50/50 [==============================] - 0s 606us/step - loss: 0.0945 - mean_squared_error: 0.0945
Epoch 22/100
50/50 [==============================] - 0s 632us/step - loss: 0.0838 - mean_squared_error: 0.0838
Epoch 23/100
50/50 [==============================] - 0s 628us/step - loss: 0.0962 - mean_squared_error: 0.0962
Epoch 24/100
50/50 [==============================] - 0s 620us/step - loss: 0.0740 - mean_squared_error: 0.0740
Epoch

...

4/4 [==============================] - 0s 2ms/step - loss: 1.9462 - mean_squared_error: 1.9462
Epoch 88/100
4/4 [==============================] - 0s 2ms/step - loss: 1.7926 - mean_squared_error: 1.7926
Epoch 89/100
4/4 [==============================] - 0s 2ms/step - loss: 1.6543 - mean_squared_error: 1.6543
Epoch 90/100
4/4 [==============================] - 0s 2ms/step - loss: 1.5964 - mean_squared_error: 1.5964
Epoch 91/100
4/4 [==============================] - 0s 2ms/step - loss: 1.4620 - mean_squared_error: 1.4620
Epoch 92/100
4/4 [==============================] - 0s 1ms/step - loss: 1.3344 - mean_squared_error: 1.3344
Epoch 93/100
4/4 [==============================] - 0s 2ms/step - loss: 1.2697 - mean_squared_error: 1.2697
Epoch 94/100
4/4 [==============================] - 0s 1ms/step - loss: 1.1488 - mean_squared_error: 1.1488
Epoch 95/100
4/4 [==============================] - 0s 2ms/step - loss: 1.1411 - mean_squared_error: 1.1411
Epoch 96/100
4/4 [=======================

4/4 [==============================] - 0s 2ms/step - loss: 7.5128 - mean_squared_error: 7.5128
Epoch 61/100
4/4 [==============================] - 0s 3ms/step - loss: 7.2887 - mean_squared_error: 7.2887
Epoch 62/100
4/4 [==============================] - 0s 2ms/step - loss: 6.8648 - mean_squared_error: 6.8648
Epoch 63/100
4/4 [==============================] - 0s 2ms/step - loss: 6.4051 - mean_squared_error: 6.4051
Epoch 64/100
4/4 [==============================] - 0s 2ms/step - loss: 6.0859 - mean_squared_error: 6.0859
Epoch 65/100
4/4 [==============================] - 0s 2ms/step - loss: 5.6961 - mean_squared_error: 5.6961
Epoch 66/100
4/4 [==============================] - 0s 2ms/step - loss: 5.3333 - mean_squared_error: 5.3333
Epoch 67/100
4/4 [==============================] - 0s 2ms/step - loss: 5.1238 - mean_squared_error: 5.1238
Epoch 68/100
4/4 [==============================] - 0s 2ms/step - loss: 4.7940 - mean_squared_error: 4.7940
Epoch 69/100
4/4 [=======================

4/4 [==============================] - 0s 2ms/step - loss: 17.3524 - mean_squared_error: 17.3524
Epoch 34/100
4/4 [==============================] - 0s 2ms/step - loss: 16.8047 - mean_squared_error: 16.8047
Epoch 35/100
4/4 [==============================] - 0s 3ms/step - loss: 16.2220 - mean_squared_error: 16.2220
Epoch 36/100
4/4 [==============================] - 0s 3ms/step - loss: 15.6749 - mean_squared_error: 15.6749
Epoch 37/100
4/4 [==============================] - 0s 2ms/step - loss: 15.0248 - mean_squared_error: 15.0248
Epoch 38/100
4/4 [==============================] - 0s 2ms/step - loss: 14.5156 - mean_squared_error: 14.5156
Epoch 39/100
4/4 [==============================] - 0s 2ms/step - loss: 13.9651 - mean_squared_error: 13.9651
Epoch 40/100
4/4 [==============================] - 0s 3ms/step - loss: 13.4714 - mean_squared_error: 13.4714
Epoch 41/100
4/4 [==============================] - 0s 2ms/step - loss: 12.9817 - mean_squared_error: 12.9817
Epoch 42/100
4/4 [=====

4/4 [==============================] - 0s 3ms/step - loss: 120.0422 - mean_squared_error: 120.0422
Epoch 8/100
4/4 [==============================] - 0s 3ms/step - loss: 57.6937 - mean_squared_error: 57.6937
Epoch 9/100
4/4 [==============================] - 0s 2ms/step - loss: 71.8584 - mean_squared_error: 71.8584
Epoch 10/100
4/4 [==============================] - 0s 2ms/step - loss: 73.1203 - mean_squared_error: 73.1203
Epoch 11/100
4/4 [==============================] - 0s 3ms/step - loss: 51.8251 - mean_squared_error: 51.8251
Epoch 12/100
4/4 [==============================] - 0s 3ms/step - loss: 42.9621 - mean_squared_error: 42.9621
Epoch 13/100
4/4 [==============================] - 0s 2ms/step - loss: 43.5485 - mean_squared_error: 43.5485
Epoch 14/100
4/4 [==============================] - 0s 3ms/step - loss: 39.0127 - mean_squared_error: 39.0127
Epoch 15/100
4/4 [==============================] - 0s 3ms/step - loss: 35.3897 - mean_squared_error: 35.3897
Epoch 16/100
4/4 [=====

4/4 [==============================] - 0s 3ms/step - loss: 0.6202 - mean_squared_error: 0.6202
Epoch 83/100
4/4 [==============================] - 0s 2ms/step - loss: 0.5242 - mean_squared_error: 0.5242
Epoch 84/100
4/4 [==============================] - 0s 3ms/step - loss: 0.4926 - mean_squared_error: 0.4926
Epoch 85/100
4/4 [==============================] - 0s 4ms/step - loss: 0.4557 - mean_squared_error: 0.4557
Epoch 86/100
4/4 [==============================] - 0s 3ms/step - loss: 0.4496 - mean_squared_error: 0.4496
Epoch 87/100
4/4 [==============================] - 0s 3ms/step - loss: 0.3939 - mean_squared_error: 0.3939
Epoch 88/100
4/4 [==============================] - 0s 3ms/step - loss: 0.3480 - mean_squared_error: 0.3480
Epoch 89/100
4/4 [==============================] - 0s 3ms/step - loss: 0.3347 - mean_squared_error: 0.3347
Epoch 90/100
4/4 [==============================] - 0s 3ms/step - loss: 0.2806 - mean_squared_error: 0.2806
Epoch 91/100
4/4 [=======================

In [45]:
%tensorboard --logdir logs/hparam_tuning